# Testing 'Concurso UPG 2024'

In [1]:
import pandas as pd

df = pd.read_csv("./Satisfacción de servicio para UPG 2024.csv", keep_default_na=False)
types = ["Open-Ended Response", "Response"]

df

,respondent_id,collector_id,date_created,date_modified,ip_address,email_address,first_name,last_name,custom_1,Nombre,...,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,¿Contratarías nuevamente nuestros servicios?.1,¿Recomendarías nuestros servicios?.1,¿Existe algo que podría ayudarnos a mejorar nuestro servicio?.1,Comparte como fue tu experiencia con tu ejecutivo de cuenta en la implementación del servicio.,"Para finalizar, nos encantaría poder presentarlos como uno más de nuestros clientes; ¿nos autorizan poder usar el nombre comercial como parte de nuestra cartera de clientes?Esto solo es por fines de referencia de nuestros servicios. Nos comprometemos a siempre guardar la confidencialidad de la información de nuestros clientes y asociados comerciales.Si tienes alguna duda, por favor pueden consultar nuestro aviso de confidencialidad.www.catch.com.mx/aviso_de_privacidad_consulting"
0,,,,,,,,,,Open-Ended Response,...,¿Qué tan profesional fue tu ejecutivo de cuenta?,¿Cómo fue la calidad del servicio de tu ejecut...,¿Cómo fue la calidad del servicio de tu ejecut...,¿Cómo fueron los tiempos de implementación y e...,¿Cómo evaluarías la efectividad de los entrega...,Response,Response,Open-Ended Response,Open-Ended Response,Response
1,7138248868,175913654,12/04/2023 12:27,10/10/2024 11:42,aa-001,,,,,Nombre del cliente,...,,,,,,,,,,"Si, autorizo que puedan utilizar el nombre de ..."
2,7147349291,175913654,10/09/2024 19:57,10/09/2024 20:37,aa-002,,,,,Prueba Catch,...,,,,,,,,,,No por el momento
3,7147281336,175913654,10/07/2024 13:51,10/07/2024 13:52,aa-003,,,,,Prueba Catch,...,,,,,,,,,,
4,7145834747,175913654,08/15/2024 01:10:17 PM,08/15/2024 01:10:50 PM,aa-004,,,,,Prueba Catch,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,6507054361,162534010,11/08/2017 16:15,11/08/2017 16:22,aa-217,,,,,Nombre del cliente,...,,,,,,,,,,
218,6489358611,162534010,10/31/2017 10:00:04 PM,10/31/2017 10:04:31 PM,aa-218,,,,,Nombre del cliente,...,,,,,,,,,,
219,6488835694,162534010,10/31/2017 03:35:25 PM,10/31/2017 03:37:32 PM,aa-219,,,,,Nombre del cliente,...,,,,,,,,,,
220,6445040337,162534257,10/11/2017 11:59,10/11/2017 12:02,aa-220,,,,,Nombre del cliente,...,,,,,,,,,,


In [2]:
for column in df.columns:
    print(column, ":", df[column].iloc[0])

respondent_id : 
collector_id : 
date_created : 
date_modified : 
ip_address : 
email_address : 
first_name : 
last_name : 
custom_1 : 
Nombre : Open-Ended Response
Compañía : Open-Ended Response
Puesto : Open-Ended Response
Correo Electrónico : Open-Ended Response
Teléfono : Open-Ended Response
¿Qué tipo de servicio te brindamos? : Response
Unnamed: 15 : Otro (especifique)
Por favor, seleccione el o los ejecutivos que les asesoran : Ana Aguirre
Unnamed: 17 : Luis Castanedo
Unnamed: 18 : Paloma Ramos
Unnamed: 19 : Rodrigo Arciniegas
Unnamed: 20 : Juan Carlos Felix
Unnamed: 21 : Luis Cadena
Unnamed: 22 : Rosalba Falcon
Unnamed: 23 : Manuel Mares
Unnamed: 24 : Miguel Flores
Unnamed: 25 : Sergio Palafox
Unnamed: 26 : Laura Naima Juárez
Unnamed: 27 : Tania Belmonte
Unnamed: 28 : Ximena Robledo
Unnamed: 29 : Victor Manuel Salgado
Unnamed: 30 : Isaac Vallejo
Unnamed: 31 : Alonso Gallardo
Unnamed: 32 : Valeria Roqueñí
Unnamed: 33 : Abril Jasso
Unnamed: 34 : Cristian Chavez
Unnamed: 35 : Angel

## Función para filtrar columnas con multiple selección

In [3]:
# Función que obtiene rangos de columnas que pertenecen a una misma pregunta con multiple selección
# Devuelve el rango de las preguntas de selección múltiple y el rango de las preguntas relacionadas
def filer_multiple_selection_columns(df):
    dictionary = {"multiple_selection" : [], "related_questions" : []}

    # Los rangos se determinan con una columna inicial y final,
    # siendo la primera columna la primera opción y la última columna la última opción
    first = None
    last = None
    last_column_type = None

    # Función interna para evaluar si se completó un rango de valores de una pregunta de selección múltiple
    def eval_last():
        nonlocal last_column_type, first, last
        if last is not None:
            # Dentro de las preguntas del formulario, la última opción de las preguntas de selección múltiple
            # es la opción "Otro (especifique)"
            if last_column_type.startswith("Otro"):
                # Agrega una tupla a la lista con el nombre de la columna inicial y final del rango de valores
                dictionary["multiple_selection"].append((first, last))
            # Si no es así, se tratan de preguntas relacionadas
            else:
                dictionary["related_questions"].append((first, last))
            first = None
            last = None

    for column in df.columns:
        # Se extrae el 'tipo' de la columna (el valor de la primera fila)
        column_type = df[column].iloc[0]
        # Las preguntas de multiple selección no son de un tipo común (Response o Open-Ended Response)
        # Más bien, su tipo suele ser el primer valor de su rango de opciones
        # Por lo que si la columna que se está examinando es común, pasa a la siguiente columna
        if column_type in types or not column_type:
            # Antes de continuar a la siguiente columna, se evalua si hay un rango completo pendiente antes
            eval_last()
            continue

        # Si NO es de tipo común (condición anterior) y no empieza con "Unnamed", quiere decir que esta
        # columna es la primera opción dentro de un rango
        # Nota: en los rangos de opciones, la columna que representa el primer valor suele tener como nombre
        # la pregunta, las demás opciones tienen un nombre vacio (pandas lo reemplaza por "Unnamed")
        if not column.startswith("Unnamed"):
            # Antes de comenzar un nuevo rango, se evalúa si hay un rango completo pendiente anterior
            eval_last()
            # Por lo tanto, se almacena el nombre de la columna como primera opción del rango
            first = column
        # Si la columna empieza con "Unnamed", quiere decir que la columna actual es parte de un rango de opciones
        else:
            # Primero se evalúa si existe un valor inicial (debería), de lo contrario seguramente esta columna sea
            # parte de otro tipo de pregunta (Pregunta de opción múltiple con opción abierta)
            if first is not None:
                # Se guardan tanto el tipo de la columna como el nombre de la columna
                # (El tipo se requiere en eval_last())
                last_column_type = column_type
                last = column

    return dictionary

In [4]:
questions_dictionary = filer_multiple_selection_columns(df)

questions_dictionary

{'multiple_selection': [('Por favor, seleccione el o los ejecutivos que les asesoran',
   'Unnamed: 36'),
  ('Por favor, seleccione el o los ejecutivos que les asesoran.1',
   'Unnamed: 52')],
 'related_questions': [('Nivel de Atención Recibida', 'Unnamed: 41'),
  ('Nivel de Atención Recibida.1', 'Unnamed: 58')]}

In [5]:
# No me agrada como quedó, pero funciona
# Obtiene la cantidad total de preguntas de selección multiple
entries_num = len(questions_dictionary["multiple_selection"])

for i in range(entries_num):
    related_questions = questions_dictionary["related_questions"]
    multiple_selection_questions = questions_dictionary["multiple_selection"]

    # Crea un dataframe con las opciones
    df_options = df.loc[:, multiple_selection_questions[i][0]:multiple_selection_questions[i][1]]
    # Crea un dataframe con las preguntas relaciondas
    df_related_questions = df.loc[:, related_questions[i][0]:related_questions[i][1]]

    # Junta las multiples opciones con las preguntas relacionadas
    new_df = pd.concat([df_options, df_related_questions], axis=1)

    column_types = new_df.iloc[0]
    new_df.columns = column_types

    new_df = new_df.drop([0]).reset_index(drop=True)

    print("DataFrame completo")
    display(new_df)

    options = df_options.iloc[0].values.flatten().tolist()

    for option in options:
        df_option = new_df[new_df[option].str.len() > 0]

        print("DataFrame de " + option)
        display(df_option)


DataFrame completo


,Ana Aguirre,Luis Castanedo,Paloma Ramos,Rodrigo Arciniegas,Juan Carlos Felix,Luis Cadena,Rosalba Falcon,Manuel Mares,Miguel Flores,Sergio Palafox,...,Valeria Roqueñí,Abril Jasso,Cristian Chavez,Angel Hernandez,Otro (especifique),¿Cómo te atendimos?,¿Qué tan profesionales fuimos?,¿Nuestro tiempo de entrega fue?,¿La calidad de nuestro servicio fue?,¿La utilidad de nuestro servicio fue?
0,,,,,,,,,Miguel Flores,,...,,,,,,Muy Bueno,Muy Bueno,Muy Bueno,Muy Bueno,
1,,,,,,,,,,,...,,Abril Jasso,,,,Muy Bueno,Muy Bueno,Excelente,Excelente,
2,,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,Luis Castanedo,,,,,,,,,...,,,,,,Muy Bueno,Excelente,Excelente,Excelente,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,,,,,,,,,,,...,,,,,,,,,,
217,,,,,,,,,,,...,,,,,,Excelente,Excelente,Excelente,Excelente,Excelente
218,,,,,,,,,,,...,,,,,,Excelente,Excelente,Muy Bueno,Excelente,Excelente
219,,,,,,,,,,,...,,,,,,Muy Bueno,Bueno,Bueno,Regular - Bueno,Bueno


DataFrame de Ana Aguirre


,Ana Aguirre,Luis Castanedo,Paloma Ramos,Rodrigo Arciniegas,Juan Carlos Felix,Luis Cadena,Rosalba Falcon,Manuel Mares,Miguel Flores,Sergio Palafox,...,Valeria Roqueñí,Abril Jasso,Cristian Chavez,Angel Hernandez,Otro (especifique),¿Cómo te atendimos?,¿Qué tan profesionales fuimos?,¿Nuestro tiempo de entrega fue?,¿La calidad de nuestro servicio fue?,¿La utilidad de nuestro servicio fue?
85,Ana Aguirre,,,,,,,,,,...,,,,,,Bueno,Regular - Malo,Muy Bueno,Excelente,
114,Ana Aguirre,,,,,,,,,,...,,,,,,Muy Bueno,Excelente,Regular - Bueno,Regular - Malo,
162,Ana Aguirre,,,,,,,,,,...,,,,,,Bueno,Muy Bueno,Muy Bueno,Muy Bueno,


DataFrame de Luis Castanedo


...

,Ana Aguirre,Rosalba Falcón,Rodrigo Arciniegas,Fernanda Roqueñí,Otro (especifique),¿Cómo fue la atención de tu ejecutivo de cuenta?,¿Qué tan profesional fue tu ejecutivo de cuenta?,¿Cómo fue la calidad del servicio de tu ejecutivo?,¿Cómo fue la calidad del servicio de tu ejecutivo?,¿Cómo fueron los tiempos de implementación y entrega del servicio?,¿Cómo evaluarías la efectividad de los entregables del servicio?


DataFrame de Fernanda Roqueñí


,Ana Aguirre,Rosalba Falcón,Rodrigo Arciniegas,Fernanda Roqueñí,Otro (especifique),¿Cómo fue la atención de tu ejecutivo de cuenta?,¿Qué tan profesional fue tu ejecutivo de cuenta?,¿Cómo fue la calidad del servicio de tu ejecutivo?,¿Cómo fue la calidad del servicio de tu ejecutivo?,¿Cómo fueron los tiempos de implementación y entrega del servicio?,¿Cómo evaluarías la efectividad de los entregables del servicio?


DataFrame de Otro (especifique)


,Ana Aguirre,Rosalba Falcón,Rodrigo Arciniegas,Fernanda Roqueñí,Otro (especifique),¿Cómo fue la atención de tu ejecutivo de cuenta?,¿Qué tan profesional fue tu ejecutivo de cuenta?,¿Cómo fue la calidad del servicio de tu ejecutivo?,¿Cómo fue la calidad del servicio de tu ejecutivo?,¿Cómo fueron los tiempos de implementación y entrega del servicio?,¿Cómo evaluarías la efectividad de los entregables del servicio?


In [6]:
for column in df.columns:
    if df[column].iloc[0] == "Response":
        print(column)

¿Qué tipo de servicio te brindamos?
¿Contratarías nuevamente nuestros servicios?
¿Recomendarías nuestros servicios?
¿Qué tipo de servicio te brindamos? 
¿Contratarías nuevamente nuestros servicios?.1
¿Recomendarías nuestros servicios?.1
Para finalizar, nos encantaría poder presentarlos como uno más de nuestros clientes; ¿nos autorizan poder usar el nombre comercial como parte de nuestra cartera de clientes?Esto solo es por fines de referencia de nuestros servicios. Nos comprometemos a siempre guardar la confidencialidad de la información de nuestros clientes y asociados comerciales.Si tienes alguna duda, por favor pueden consultar nuestro aviso de confidencialidad.www.catch.com.mx/aviso_de_privacidad_consulting


In [7]:
def eval_last(first, last, last_column_type):
    if first is not None and last is not None:
        if last_column_type.startswith("Otro"):
            return "multiple_selection"
        else:
            return "related_questions"

    return None


def get_column_range(df: pd.DataFrame, column_name: str, get_consecutive_related_questions: bool) -> tuple:
    result = {"multiple_selection": None, "related_questions": None}
    first = None
    last = None
    last_column_type = None

    for column in df.loc[:, column_name:].columns:
        column_type = df[column].iloc[0]
        # Si esto es verdad, la pregunta no es de multiple selección
        if column_type in ["Open-Ended Response", "Response"] or not column_type:
            range_type = eval_last(first, last, last_column_type)
            if range_type:
                result[range_type] = (first, last)
            return result
        elif not column.startswith("Unnamed"):
            range_type = eval_last(first, last, last_column_type)
            if range_type is not None:
                result[range_type] = (first, last)

            if get_consecutive_related_questions:
                first = column
            else:
                return result
        else:
            if first is not None:
                last = column
                last_column_type = column_type

    return None

In [19]:
result = get_column_range(df, "Por favor, seleccione el o los ejecutivos que les asesoran", True)

print(result["multiple_selection"])
print(result["related_questions"])

('Por favor, seleccione el o los ejecutivos que les asesoran', 'Unnamed: 36')
('Nivel de Atención Recibida', 'Unnamed: 41')
